In [1]:
import pandas as pd
#import sklearn as sk
import numpy as np
import xgboost as xgb

In [2]:
train_df = pd.read_csv("/Users/Stefano_1/Downloads/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/Users/Stefano_1/Downloads/house-prices-advanced-regression-techniques/test.csv")


In [55]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline

In [4]:
X = train_df.drop('SalePrice', axis = 1)
y = np.log(train_df['SalePrice'])

In [5]:
for column in X.select_dtypes(include=['object']).columns:
    X[column] = X[column].astype('category')

In [6]:
for column in test_df.select_dtypes(include=['object']).columns:
    test_df[column] = test_df[column].astype('category')

In [7]:
X.select_dtypes(include=['int64']).columns

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'],
      dtype='object')

In [61]:
#numerical columns pipeline
num_transformer = Pipeline([('imputer', KNNImputer(n_neighbors=2)),
                            ('scaler', StandardScaler())])

In [62]:
#categorical columns pipeline
cat_transformer = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                            ('encoder', OneHotEncoder(handle_unknown='ignore'))])

In [63]:
cat_cols = X.select_dtypes(include=['object', 'category']).columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
prep = make_column_transformer((num_transformer, num_cols),
                               (cat_transformer, cat_cols),
                               remainder= 'passthrough')

In [64]:
#from sklearn.linear_model import LassoCV
#from sklearn.ensemble import AdaBoostRegressor
#from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import VotingRegressor
#from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [65]:
param_grid = {
    'learning_rate': [0.005,0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300,400,500],
    'max_depth': [3, 4, 5]
}

In [66]:
mod = XGBRegressor()

grid = GridSearchCV(mod,
                        param_grid=param_grid,
                        cv=5,
                        scoring='neg_mean_squared_error',
                        verbose=2)

pipeline = Pipeline([('preprocessor', prep),
                    ('regressor', grid)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)
pipeline.fit(X_train, y_train)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=100; total time=   0.1s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=200; total time=   0.3s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=200; total time=   0.2s
[CV] END .learning_rate=0.005, max_depth=3, n_estimators=300; total time=   0.3s
[CV] END .learning_rate=0.005, max_depth=3, n_e

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=2)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsm...
                                                     max_delta_step=None,
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...),
                              param_grid={'learning_rate': [0.005, 0.01, 0.1,
                                                            0.2],
                                          'max_depth': [3, 4, 5],
                                          'n_estimators': [100, 200, 300, 400,
                                                           500]},
                              scoring='neg_mean_squared_error', verbose=2))])

In [67]:
grid.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [72]:
y_pred = pipeline.predict(X_test)

In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [73]:
mean_squared_error(y_pred, y_test, squared= False)

/opt/homebrew/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


0.16107814353315314

In [74]:
test_df['SalePrice'] = pipeline.predict(test_df)
test_df['SalePrice'] = np.exp(test_df['SalePrice'])

In [75]:
test_df[['Id', 'SalePrice']].to_csv('/Users/Stefano_1/Downloads/house-prices-advanced-regression-techniques/subs/xgb_pred_log_fe2.csv', index_label= 'Id', index = False)

In [ ]:
# reg_lasso = LassoCV()
# alpha_values = np.arange(0.1, 1, 0.1)
# #lasso_params = {'alpha' : alphas}
# #models = [('lasso_cv', LassoCV()), ]

# #clf = GridSearchCV(reg_lasso, lasso_params)
# clf = LassoCV(alphas= alpha_values)
# clf.fit(train_df, train_values)
# lasso_preds = clf.predict(test_df)

In [ ]:
# reg_ada = AdaBoostRegressor()

# ada_params = {'n_estimators' : [50, 100, 500],
#               'loss' : ['square', 'linear']}
# clf = GridSearchCV(reg_ada, ada_params)
# clf.fit(train_df, train_values)
# clf.score()

In [ ]:
reg_tree = RandomForestRegressor()
X_train, X_test, y_train, y_test = train_test_split(train_df, train_values, test_size=0.2, random_state=16)
reg_tree.fit(X_train, y_train)
reg_tree.score()

In [ ]:
import xgboost as xgb
reg_xg = xgb.XGBRegressor(enable_categorical=True)

X_train, X_test, y_train, y_test = train_test_split(train_df, train_values, test_size=0.2, random_state=16)
dtrain = xgb.DMatrix(X_train, label= y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label = y_test, enable_categorical=True)
reg_xg.fit(train_df, train_values)
reg_xg.predict(test_df)
reg_xg.score()

In [ ]:
train_df.dtypes